<a href="https://colab.research.google.com/github/Yashvishe13/Data-Science/blob/main/Collecting_data_from_Youtube_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing Libraries
import requests
import pandas as pd
import time

In [2]:
#Keys
API_KEY = 'xxxxxx'
CHANNEL_ID = 'xxxxxx'

In [3]:
#Get video statistics
def get_video_stats(video_id):
  #Second API call
  url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
  response_video_stats = requests.get(url_video_stats).json()
  # Views Count
  try:
    view_counts = response_video_stats['items'][0]['statistics']['viewCount']
  except:
    view_counts = 'NULL'  
  # Likes Count
  try:
    like_counts = response_video_stats['items'][0]['statistics']['likeCount']
  except:
    like_counts = 'NULL'
  # Comments Count
  try:
    comments_counts = response_video_stats['items'][0]['statistics']['commentCount']
  except:
    comments_counts = 'NULL'
  return view_counts, like_counts, comments_counts

In [4]:
def get_video(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","&")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_stats(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [5]:
# Main
df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])
df = get_video(df)
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,hiehLFrTlRs,20 WOMEN VS 1 SIDEMEN: KSI EDITION,2021-12-19,6557230,426924,18920
1,7Oa-GtZE9tE,"SIDEMEN $100,000 HOT VS COLD HOLIDAY",2021-12-12,9138935,458962,19389
2,GfWL0-dI3Dc,SIDEMEN PUB GOLF (EUROPE EDITION),2021-12-05,7343433,373640,11612
3,DSEnADJ-tJ0,SIDEMEN HIDE & SEEK IN THE WORLD&#39;S BIGGEST...,2021-11-28,6754660,327904,9400
4,zijF6-hXVdQ,SIDEMEN EXTREME ROBOT WARS (BATTLE BOTS),2021-11-21,5779436,335525,9546
...,...,...,...,...,...,...
196,-cE-1EDV2Xs,SIDEMEN PRO CLUBS IS BACK!,2017-01-08,4087811,157535,7366
197,jXKOWvsz1To,THE SIDEMEN AWARDS,2016-10-09,2889811,83361,6885
198,J_NniTO22_A,SIDEMEN FC VS YOUTUBE ALLSTARS CHARITY FOOTBAL...,2016-06-03,25240890,434634,36579
199,5j_fRfbscaE,SIDEMEN FACE PAINTING CHALLENGE,2016-05-28,4081149,134269,16426


In [7]:
df.isnull().sum()

video_id         0
video_title      0
upload_date      0
view_count       0
like_count       0
comment_count    0
dtype: int64